# Install required packages

In [1]:
!pip install pandas nltk textblob python-docx
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 1.3 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [2]:
import pandas as pd
import nltk
from textblob import TextBlob
from docx import Document
from collections import defaultdict
from google.colab import files

#  Download NLTK resources

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
# Directly read the uploaded file from the Colab filesystem
df = pd.read_csv("/content/Company Ticker Progress 26th July.csv")

# Optional: preview the data
df.head()

,Ticker,Title,Summary,URL,Original Published Time,Datetime,News Age (Days),News Time,Current Price,1 Hour Ago,4 Hours Ago,Previous Day Close,7 Days Ago,1 Hour Ago % Change,4 Hours Ago % Change,Previous Day Close % Change,7 Days Ago % Change,Headline Sentiment,Summary Sentiment
0,PFE,Metastatic Hormone Refractory Prostate Cancer ...,"The report details drug profiles and phases, h...",https://finance.yahoo.com/news/metastatic-horm...,"Fri, 25 Jul 2025 14:40:00 +0000",2025-07-25 10:40:00-04:00,1,2025-07-25 10:40:00-04:00,24.69,24.69,24.69,25.36,24.55,0.00,0.00,-2.64,0.57,0.00,-0.72
1,PFE,Pfizer Inc. (PFE) Is a Trending Stock: Facts t...,Zacks.com users have recently been watching Pf...,https://finance.yahoo.com/news/pfizer-inc-pfe-...,"Fri, 25 Jul 2025 13:00:05 +0000",2025-07-25 09:00:05-04:00,1,2025-07-25 09:00:05-04:00,24.69,24.69,24.69,25.42,24.58,0.00,0.00,-2.87,0.45,0.00,0.48
2,MRNA,"Biotech Is Booming, and This Undervalued REIT ...",While most commercial office landlords are sti...,https://www.fool.com/investing/2025/07/25/biot...,"Fri, 25 Jul 2025 21:36:00 +0000",2025-07-25 17:36:00-04:00,1,2025-07-25 17:36:00-04:00,34.14,34.14,34.31,34.01,31.25,0.00,-0.50,0.38,9.25,0.00,0.38
3,MRNA,"Stocks to watch next week: Microsoft, Apple, S...",Earnings preview of key companies reporting in...,https://uk.finance.yahoo.com/news/microsoft-ap...,"Fri, 25 Jul 2025 14:36:32 +0000",2025-07-25 10:36:32-04:00,1,2025-07-25 10:36:32-04:00,33.63,33.95,33.95,34.37,31.75,-0.94,-0.94,-2.15,5.92,0.00,0.00
4,MRNA,Moderna (MRNA) Expected to Beat Earnings Estim...,Moderna (MRNA) possesses the right combination...,https://finance.yahoo.com/news/moderna-mrna-ex...,"Fri, 25 Jul 2025 14:00:08 +0000",2025-07-25 10:00:08-04:00,1,2025-07-25 10:00:08-04:00,33.63,33.95,33.95,34.37,31.75,-0.94,-0.94,-2.15,5.92,0.07,0.23


In [6]:
# Define known financial keyword categories
KEYWORD_CATEGORIES = {
   "earnings": ["earnings", "EPS", "profit", "revenue", "quarterly", "beating estimates", "missed estimates"],
    "acquisition": ["acquisition", "buyout", "purchase", "takeover", "acquire", "acquiring"],
    "merger": ["merger", "combine", "merge", "consolidation", "integration"],
    "FDA approval": ["FDA", "approval", "approved", "authorized", "clearance", "clinical trial", "phase 3", "drug"],
    "stimulus": ["stimulus", "aid", "relief", "package", "subsidy", "government support"],
    "dividend": ["dividend", "payout", "distribution", "dividend hike", "dividend cut"],
    "stock split": ["stock split", "reverse split", "share split"],
    "layoffs": ["layoff", "job cut", "redundancy", "fired", "downsizing", "workforce reduction"],
    "lawsuit": ["lawsuit", "legal", "settlement", "class action", "sued", "litigation"],
    "partnership": ["partnership", "collaboration", "strategic alliance", "joint venture"],
    "guidance": ["guidance", "forecast", "outlook", "expectations", "projection"],
    "innovation": ["launch", "innovation", "product release", "technology", "patent", "upgrade"],
    "bankruptcy": ["bankruptcy", "chapter 11", "default", "insolvency", "debt restructure"],
    "analyst rating": ["upgrade", "downgrade", "analyst", "target price", "buy rating", "sell rating"],
    "environmental": ["sustainability", "carbon", "green", "climate", "ESG", "environmental"],
    "cybersecurity": ["hack", "data breach", "cyberattack", "security flaw", "ransomware"]
}

# Detect known keywords in a text
def detect_keywords(text):
    detected = []
    for category, words in KEYWORD_CATEGORIES.items():
        for word in words:
            if word.lower() in text.lower():
                detected.append(category)
                break  # One match per category is enough
    return detected or ['Other']

# Storage dictionary
company_keywords = defaultdict(lambda: {'positive': [], 'negative': []})

# 🔍 Process each row to extract keywords & sentiment
for _, row in df.iterrows():
    ticker = row['Ticker']
    summary = str(row['Summary'])  # Ensure it's a string

    blob = TextBlob(summary)
    sentiment = 'positive' if blob.sentiment.polarity >= 0 else 'negative'

    detected = detect_keywords(summary)

    company_keywords[ticker][sentiment].extend(detected)

for ticker in company_keywords:
    for sentiment in ['positive', 'negative']:
        keywords = list(set(company_keywords[ticker][sentiment]))
        company_keywords[ticker][sentiment] = sorted(keywords)

# Create the Word document
doc = Document()
doc.add_heading('Company Progress Keyword Summary', 0)

for ticker, sentiment_data in sorted(company_keywords.items()):
    doc.add_heading(ticker, level=1)

    pos_keywords = ', '.join(sentiment_data['positive']) or "No data"
    neg_keywords = ', '.join(sentiment_data['negative']) or "No data"

    doc.add_paragraph(f"✅ Positive Keywords: {pos_keywords}")
    doc.add_paragraph(f"❌ Negative Keywords: {neg_keywords}")
    doc.add_paragraph("")  # Spacer

#  Save and download the DOCX

In [7]:
output_file = "Company_Progress_Summary.docx"
doc.save(output_file)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>